In [1]:
#import boto3
from astropy.io import fits
from io import BytesIO
import sys
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
sys.path.append('/home/bekah/gPhoton2')
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet


In [2]:
def centile_clip(image, centiles=(0, 90)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    import numpy as np
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)

    if isinstance(image, np.ma.MaskedArray):
        return result

    return result.data

In [ ]:
# dose maps are made using what from photonlists? 

In [ ]:
# import mask at 2400 wide resolution 
# this was made with dose maps 
fuv_drawn = fits.open("/home/bekah/glcat/masks/drawn/fuv_drawn_mask.fits")

In [ ]:
# read in photonlist 
photons = parquet.read_table("/home/bekah/glcat/notebooks/masks_flats/e00834-fd-b00.parquet").to_pandas()

In [ ]:
photons = photons[(photons['detrad'] < 600)]

In [ ]:
plt.imshow(fuv_drawn[0].data,interpolation=None)

In [ ]:
def dosemap_ranges(radius):
    slop = 400 - radius
    return [slop, 3200 - slop], [slop, 3200 - slop]

x, y = dosemap_ranges(600)

In [ ]:
x

In [ ]:
photons['col_rounded'] = photons['col'].round().astype(int)*4 # x 
photons['row_rounded'] = photons['row'].round().astype(int)*4 # y 

In [ ]:
photons = photons[(photons['col_rounded'] >= -200) & (photons['row_rounded'] <= 3400) & (photons['row_rounded'] >= -200) & (photons['col_rounded'] <= 3400)]

In [ ]:
plt.imshow(fuv_drawn[0].data,interpolation=None, origin='lower', cmap='grey')
plt.scatter(photons['col']*(16/3)+200*(4/3),photons['row']*(16/3)+200*(4/3),s=.00005)


In [ ]:
mask = fuv_drawn[0].data[photons['col'], photons['row']] == 1
photons['is_one'] = mask

In [ ]:
plt.scatter(photons['row']*(16/3)+200*(4/3),photons['col']*(16/3)+200*(4/3),s=.00005)


In [ ]:
3316*(4/3)+200

In [ ]:
(photons['row']*(16/3)+200*(4/3)).describe()

In [ ]:
-*4/3

In [ ]:
(photons['col']).describe()

In [ ]:
-200*(4/3)

In [ ]:
plt.scatter((photons['col_rounded']*4/3),(photons['row_rounded']*4/3))

In [ ]:
plt.scatter((photons['col_rounded']*4/3)+300,(photons['row_rounded']*4/3)+300)

In [ ]:
nf = parquet.read_table("/home/bekah/gPhoton2/test_data/e23456/e23456-nd-b00.parquet").to_pandas()

In [ ]:
plt.scatter(nf['y'],nf['detrad'],s=.5)

In [ ]:
plt.scatter(nf['row'],nf['col'],c=nf['detrad'],s=.5,cmap='grey')

In [ ]:
nf['col_rounded'] = nf['col'].astype(int)*4 # x 
nf['row_rounded'] = nf['row'].astype(int)*4 # y 

In [ ]:
nf = nf[(nf['col_rounded'] >= -200) & (nf['row_rounded'] <= 3400) & (nf['row_rounded'] >= -200) & (nf['col_rounded'] <= 3400)]

In [ ]:
plt.imshow(nuv_drawn[0].data,interpolation=None, origin='lower')
plt.scatter(nf['row']*(16/3)+200*(4/3),nf['col']*(16/3)+200*(4/3),s=.00005)


In [ ]:
plt.hist(nf['row'])

In [ ]:
(nf['col']*(16/3)+200*(4/3)).describe()

In [ ]:
photons['col'].describe()

In [3]:
nuv_drawn = fits.open("/media/bekah/BekahA/hotspot/drawn/nuv_spots.fits")

FileNotFoundError: [Errno 2] No such file or directory: '/media/bekah/BekahA/hotspot/drawn/nuv_spots.fits'

In [ ]:
photons['row'].describe()

In [ ]:
plt.hist(photons['row'])

In [ ]:
nuv_im = fits.open("/home/bekah/gPhoton2/test_data/e23456-nd-b00-f3000-t00000-g_dose.fits.gz")

In [ ]:
nf = parquet.read_table("/home/bekah/gPhoton2/test_data/e23456/e23456-nd-b00.parquet").to_pandas()

In [ ]:
nf = nf[(nf['row']*4 >= -200) & (nf['row']*4 <= 3400) & (nf['col']*4 >= -200) & (nf['col']*4 <= 3400)]

In [ ]:
plt.scatter(nf['row']*(16/3)+200*(4/3),nf['col']*(16/3)+200*(4/3),s=.00005, c='red',alpha=1)
plt.imshow(centile_clip(nuv_im[0].data), interpolation=None, origin='lower',alpha=.5)
plt.savefig('overlay.png')

In [ ]:
plt.imshow(centile_clip(nuv_im[0].data), origin='lower')


In [ ]:
plt.subplots(figsize=(10, 10))
plt.scatter(nf['col']*(16/3)+200*(4/3),nf['row']*(16/3)+200*(4/3),s=.00005, c='red',alpha=.2)
plt.imshow(centile_clip(nuv_im[0].data), interpolation=None, origin='lower',alpha=.5)

In [ ]:
# ok so I think I've finally figured out how to match up col and row to the dose maps lmao it only took several days 
nf['col']*(16/3)+200*(4/3),nf['row']*(16/3)+200*(4/3)